In [1]:
%pip install -qU fastapi uvicorn aiortc

Note: you may need to restart the kernel to use updated packages.


To make fastapi run in jupyter

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from aiortc import RTCPeerConnection, RTCSessionDescription, RTCIceCandidate
from aiortc.contrib.media import MediaPlayer, MediaRelay

In [4]:
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],  # Allow all HTTP methods
    allow_headers=["*"],  # Allow all headers
)

relay = MediaRelay()
pc = RTCPeerConnection()

In [ ]:
@app.post("/api/webrtc/offer")
async def webrtc_offer(request: Request):
    data = await request.json()
    offer = data.get("sdp")

    # Set the remote SDP (offer)
    await pc.setRemoteDescription(RTCSessionDescription(sdp=offer, type="offer"))

    # Add media processing logic here if needed
    @pc.on("track")
    async def on_track(track):
        print(f"Received track: {track.kind}")
        if track.kind == "audio":
            player = MediaPlayer(track)
            relay.subscribe(player)

    # Create an SDP answer
    answer = await pc.createAnswer()
    await pc.setLocalDescription(answer)

    return {"answer": pc.localDescription.sdp}


@app.post("/api/webrtc/ice-candidate")
async def add_ice_candidate(request: Request):
    data = await request.json()
    candidate = data.get("candidate")
    sdpMid = data.get("sdpMid")
    sdpMLineIndex = data.get("sdpMLineIndex")

    if candidate and sdpMid is not None and sdpMLineIndex is not None:
        ice_candidate = {
            "candidate": candidate,
            "sdpMid": sdpMid,
            "sdpMLineIndex": sdpMLineIndex,
        }
        await pc.addIceCandidate(ice_candidate)

In [ ]:
if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [299631]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:47600 - "POST /api/webrtc/ice-candidate HTTP/1.1" 200 OK
INFO:     127.0.0.1:47612 - "POST /api/webrtc/ice-candidate HTTP/1.1" 200 OK
INFO:     127.0.0.1:47618 - "POST /api/webrtc/ice-candidate HTTP/1.1" 200 OK
INFO:     127.0.0.1:47628 - "POST /api/webrtc/ice-candidate HTTP/1.1" 200 OK
INFO:     127.0.0.1:47628 - "POST /api/webrtc/ice-candidate HTTP/1.1" 200 OK
INFO:     127.0.0.1:47618 - "POST /api/webrtc/ice-candidate HTTP/1.1" 200 OK
INFO:     127.0.0.1:47612 - "POST /api/webrtc/ice-candidate HTTP/1.1" 200 OK
INFO:     127.0.0.1:47600 - "POST /api/webrtc/ice-candidate HTTP/1.1" 200 OK
INFO:     127.0.0.1:47586 - "POST /api/webrtc/offer HTTP/1.1" 200 OK


In [ ]:
pc.setLocalDescription

<bound method RTCPeerConnection.setLocalDescription of <aiortc.rtcpeerconnection.RTCPeerConnection object at 0x7ea479720b90>>